<a href="https://colab.research.google.com/github/eskayML/free-ai-classes-2023/blob/main/Free_AI_Classes_Starter_NoteBook_by_eskayML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINAL COPY PUSHED ON 7TH OF AUGUST (after the close of the competition)
zindi username: **eskayML**

In [1]:
!pip install catboost optuna -q
!unzip /content/free-ai-classes-in-every-city-hackathon-2023.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00
Archive:  /content/free-ai-classes-in-every-city-hackathon-2023.zip
  inflating: Sample_submission.csv   
  inflating: Housing_dataset_train.csv  
  inflating: Housing_dataset_test.csv  


## 1. IMPORT REQUIRED PACKAGES/ LIBRARIES

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import  train_test_split,cross_val_score, cross_val_predict,KFold
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler , PowerTransformer
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import VotingRegressor


import warnings
warnings.filterwarnings('ignore')

# 2. LOAD DATA

In [3]:
train = pd.read_csv('Housing_dataset_train.csv')
test = pd.read_csv('Housing_dataset_test.csv')
sample_sub = pd.read_csv('Sample_submission.csv')

In [4]:
train.isna().sum() , test.isna().sum()

(ID                  0
 loc              1813
 title            1722
 bedroom          1799
 bathroom         1805
 parking_space    1811
 price               0
 dtype: int64,
 ID               0
 loc              0
 title            0
 bedroom          0
 bathroom         0
 parking_space    0
 dtype: int64)

In [5]:
train.head()

,ID,loc,title,bedroom,bathroom,parking_space,price
0,3583,Katsina,Semi-detached duplex,2.0,2.0,1.0,1149999.565
1,2748,Ondo,Apartment,NaN,2.0,4.0,1672416.689
2,9261,Ekiti,NaN,7.0,5.0,NaN,3364799.814
3,2224,Anambra,Detached duplex,5.0,2.0,4.0,2410306.756
4,10300,Kogi,Terrace duplex,NaN,5.0,6.0,2600700.898


In [6]:
train.columns.tolist()

['ID', 'loc', 'title', 'bedroom', 'bathroom', 'parking_space', 'price']

In [7]:
# sns.pairplot(train)

In [8]:
train = train.dropna(subset = ['loc','title'])

In [9]:
train.title.value_counts()

Flat                    1182
Apartment               1147
Townhouse               1139
Semi-detached duplex    1133
Mansion                 1125
Detached duplex         1115
Penthouse               1103
Bungalow                1102
Terrace duplex          1095
Cottage                  390
Name: title, dtype: int64

# 4. DATA PREPROCESSING




In [10]:
all = pd.concat([train, test], ignore_index = False)

In [11]:
all['log_price'] = np.log1p(all['price'])
all['total_rooms'] = all['bedroom'] + all['bathroom']


all['has_parking'] = all['parking_space'].apply(lambda x:  x > 0  )
all['is_duplex'] = all['title'].astype(str).apply(lambda x: 'duplex' in x)

all['bed_bath_interaction'] = all['bedroom'] * all['bathroom']
all['bed_parking_interaction'] = all['bedroom'] * all['parking_space']
all['bath_parking_interaction'] = all['bathroom'] * all['parking_space']

all['loc_mean_log_price'] = all.groupby('loc')['log_price'].transform('mean')
all['loc_std_log_price'] = all.groupby('loc')['log_price'].transform('std')

all['title_mean_log_price'] = all.groupby('title')['log_price'].transform('mean')
all['title_max_log_price'] = all.groupby('title')['log_price'].transform('max')
all['title_std_log_price'] = all.groupby('title')['log_price'].transform('std')

all['loc_avg_bedroom'] = all.groupby('loc')['total_rooms'].transform('mean')
all['loc_max_bathroom'] = all.groupby('loc')['bathroom'].transform('max')

all['location_length'] = all['loc'].astype(str).apply(len)
all['title_length'] = all['title'].astype(str).apply(len)
all['parking_ratio'] = all['parking_space'] / (all['bedroom'] + all['bathroom'])
all['total_rooms_bin'] = all['total_rooms'] // 2


def create_bin(column):
  if column in range(1,3):
    return '1-2'
  elif column in range(3,5):
    return '3-4'
  elif column in range(5,7):
    return '5-6'
  else:
    return 'higher'

all['bedroom_bin']  = all.bedroom.apply(create_bin)
all['bathroom_bin']  = all.bathroom.apply(create_bin)
all['parking_bin']  = all.parking_space.apply(create_bin)

all = all[all.columns.difference(['loc','title'])]
all = pd.get_dummies(all, columns = ['bedroom_bin','bathroom_bin','parking_bin'])

In [12]:
all.shape

(16531, 35)

In [13]:
train, test = all[all.price.notna()] , all[all.price.isna()]

In [14]:

train  = train[~((train['price'] > 800000) & ( train['parking_space'] <= 3) & (train['bedroom'] <= 3) & (train['bathroom']<=3) )  ]
train = train[~((train['log_price'] > 16) & (train['total_rooms'] < 8))]

In [15]:
X,y = train.drop(['ID','price','log_price'],axis = 1), train['price']
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 42, test_size = .2) # best 'random state = 42'

In [16]:
train.shape

(9431, 35)

# 5. TRAINING A MODEL


In [17]:
lg_params = {'subsample':.9,'max_depth': 2, 'learning_rate': 0.12169756811695075, 'max_iter': 284, 'min_child_weight': 8, 'l2_regularization': 0.047478207817647555, 'max_leaf_nodes': 90, 'max_bins': 81, 'random_state':0}
cv_scores = cross_val_score(LGBMRegressor(**lg_params), X_train,y_train, cv = 5,scoring = 'neg_mean_squared_error', n_jobs = -1 )
np.sqrt(-cv_scores.mean())

385158.18174465705

In [18]:

cb_params = {
    'silent':True,
    'random_state':0,
    'max_depth':2,
    'min_child_samples':5,
    'learning_rate':.118,
}

cv_scores = cross_val_score(CatBoostRegressor(**cb_params), X_train,y_train, cv = 5,scoring = 'neg_mean_squared_error', n_jobs = -1 )
np.sqrt(-cv_scores.mean())


385274.7575631086

In [19]:
hist_params = {'random_state':0,'max_depth': 2, 'learning_rate': 0.05194253666954146, 'max_iter': 970, 'min_samples_leaf': 5, 'l2_regularization': 0.043711911345217194, 'max_leaf_nodes': 10, 'max_bins': 73}
cv_scores = cross_val_score(HistGradientBoostingRegressor(**hist_params), X_train,y_train, cv = 5,scoring = 'neg_mean_squared_error', n_jobs = -1 )
np.sqrt(-cv_scores.mean())

385904.3513937716

In [20]:
cat = CatBoostRegressor(**cb_params)
hist = HistGradientBoostingRegressor(**hist_params)
lg = LGBMRegressor(**lg_params)

ensemble = VotingRegressor([('cat',cat),('hist',hist),('lg',lg)],weights = [4,3,2])
ensemble.fit(X,y)
ensemble_preds= ensemble.predict(X_test)
mean_squared_error(y_test, ensemble_preds, squared = False) #rmse

[LightGBM] [Warning] lambda_l2 is set with l2_regularization=0.047478207817647555, will be overridden by reg_lambda=0.0. Current value: lambda_l2=0.0
[LightGBM] [Warning] num_leaves is set=31, max_leaf_nodes=90 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_iterations is set=284, max_iter=284 will be ignored. Current value: num_iterations=284


400410.13666149613

In [21]:
# import warnings,optuna
# warnings.filterwarnings('ignore')


# def objective(trial):
#     # Define the hyperparameters to tune
#     params = {
#         'max_depth': 2,
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'max_iter': trial.suggest_int('max_iter', 100, 1000),
#         'min_child_samples': trial.suggest_int('min_child_weight', 1, 10),
#         'reg_lambda': trial.suggest_float('l2_regulariztion', 0.0, 0.1),
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),
#         'random_state':0,
#         'max_bins':trial.suggest_int('max_bins', 20,200),
#         # 'silent':True,
#         'subsample':trial.suggest_float('subsample', 0.5, 1.0),
#     }

#     # Create the CatBoost classifier with the suggested hyperparameters
#     clf = LGBMRegressor(**params)

#     # Evaluate using cross-validation
#     scores = cross_val_score(clf,X_train,y_train, cv=5,n_jobs = -1, scoring = 'neg_mean_squared_error')  # Replace X and y with your data
#     # Return the mean cross-validation score as the objective value
#     return np.sqrt(-scores.mean())


# # Create the Optuna study
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Print the best hyperparameters and objective value
# best_params = study.best_params
# best_score = study.best_value
# print("Best Hyperparameters: ", best_params)
# print("Best Objective Value: ", best_score)

In [22]:
sub_file = pd.DataFrame({
    'ID': test.ID,
    'price': ensemble.predict(test[X.columns])
})

sub_file.to_csv('final ensemble.csv', index = False)